<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20221209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* Customer Analysis
* Iterators

In [1]:
import pandas as pd 
import numpy as np
import plotnine as ggplot
from plotnine import ggplot, aes 
import matplotlib.pyplot as plt 
import seaborn as sns 
from datetime import datetime, date, time, timezone, timedelta 
import os
import re

from functools import reduce 
from collections import defaultdict

# Customer Analysis

In [2]:
os.listdir()

['.config',
 'ecommerce-data.csv',
 '1_order_payments_dataset.csv',
 'scanner_data.csv',
 'sales_vs_discount.csv',
 'online_retail_II.xlsx',
 'KPMG_VI_New_raw_data_update_final.csv',
 'online_retail.xlsx',
 'sample-sales-data.csv',
 'mall_customers.csv',
 '1_customers_dataset.csv',
 'groceries.csv',
 'mock_sales_data.csv',
 'sales.csv',
 'match_result_test.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 '1_orders_dataset.csv',
 'sales_data_with_stores.csv',
 'bz_data_analyis.csv',
 'test.csv',
 'sample_data']

In [11]:
df = pd.read_csv('ecommerce-data.csv', encoding = 'unicode_escape')
df.columns = ['_'.join(re.split('(?=[A-Z][a-z])', a[0].lower() + a[1:])).lower() for a in df.columns]
df = df[(~df.invoice_no.astype(str).str.contains('C')) & (df.country == 'United Kingdom')]
df['invoice_date'] = pd.to_datetime(df['invoice_date'], infer_datetime_format=True)
df['date'] = df['invoice_date'].to_numpy().astype('datetime64[D]')
df['week'] = df['date'] - df['date'].dt.weekday.astype('timedelta64[D]')
df['rev'] = df.quantity * df.unit_price
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country,date,week,rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-11-29,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-11-29,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34


In [21]:
cnt = df.groupby(['customerid', 'week'])['rev'].agg(['sum','count']).reset_index()
cnt.columns = ['id', 'week', 'sum', 'count']
cnt.head()

,id,week,sum,count
0,12346.0,2011-01-17,77183.60,1
1,12747.0,2010-11-29,358.56,7
2,12747.0,2010-12-13,347.71,8
3,12747.0,2011-01-17,303.04,5
4,12747.0,2011-02-28,310.78,12


In [28]:
info_m = pd.pivot_table(cnt, index = 'id', columns = 'week', values = 'sum').reset_index()
info_m = pd.melt(info_m, id_vars = 'id', var_name = 'week', value_name = 'money').sort_values(['id','week'], ascending = [1,0])
info_m = info_m.groupby(['id']).head(3)
info_m = info_m.groupby(['id'])['money'].sum().reset_index()
info_m.head()

,id,money
0,12346.0,0.00
1,12747.0,438.50
2,12748.0,5576.04
3,12749.0,763.06
4,12820.0,210.35


In [29]:
info_f = pd.pivot_table(cnt, index = 'id', columns = 'week', values = 'count').reset_index()
info_f = pd.melt(info_f, id_vars = 'id', var_name = 'week', value_name = 'freq').sort_values(['id','week'], ascending = [1,0])
info_f = info_f.groupby(['id']).head(3)
info_f = info_f.groupby(['id'])['freq'].sum().reset_index()
info_f.head()

,id,freq
0,12346.0,0.0
1,12747.0,7.0
2,12748.0,1165.0
3,12749.0,39.0
4,12820.0,12.0


In [31]:
date_max = df['week'].max()
date_max

Timestamp('2011-12-05 00:00:00')

In [34]:
info_r = cnt.groupby(['id']).agg({'week' : lambda x : date_max - x.max()}).reset_index().rename(columns = {'week':'recency'})
info_r['recency'] = info_r['recency'].dt.days
info_r.head()

,id,recency
0,12346.0,322
1,12747.0,0
2,12748.0,0
3,12749.0,0
4,12820.0,0


In [36]:
info = pd.merge(pd.merge(info_r, info_f, 'left', 'id'), info_m, 'left', 'id')
info = info.assign(r = pd.qcut(info['recency'], 5, list('12345')),
                   f = pd.qcut(info['freq'].rank(method = 'first'), 5, list('54321')),
                   m = pd.qcut(info['money'].rank(method = 'first'), 5, list('54321'))
                   )
info.head()

,id,recency,freq,money,r,f,m
0,12346.0,322,0.0,0.00,5,5,5
1,12747.0,0,7.0,438.50,1,2,1
2,12748.0,0,1165.0,5576.04,1,1,1
3,12749.0,0,39.0,763.06,1,1,1
4,12820.0,0,12.0,210.35,1,1,1


# Iterators

In [37]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M',
  'age': 73},
 {'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F',
  'age': 29},
 {'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M',
  'age': 51},
 {'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F',
  'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]

In [38]:
{k : eval(f'reduce(lambda acc, cur : acc + [cur["{k}"]], users, [])') for k in list(users[0].keys())}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}